In [ ]:
# | default_exp cli.user

In [ ]:
from airt.testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.


In [ ]:
# | export

from typing import *

In [ ]:
# | exporti

import datetime as dt
import os

import pandas as pd
import qrcode
import typer
from tabulate import tabulate
from typer import echo

from airt.cli import helper
from airt.client import Client
from airt.constant import SERVICE_PASSWORD
from airt.logger import get_logger, set_level

In [ ]:
import logging
from contextlib import contextmanager
from random import randrange

import pytest
from typer.testing import CliRunner

import airt.sanitizer
from airt.client import Client, User
from airt.constant import (
    SERVICE_PASSWORD,
    SERVICE_SUPER_USER,
    SERVICE_TOKEN,
    SERVICE_USERNAME,
)

In [ ]:
# | exporti

app = typer.Typer(
    help="A set of commands for managing users and their authentication in the server."
)

In [ ]:
runner = CliRunner()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
set_level(logging.WARNING)

In [ ]:
# Testing logger settings

display(logger.getEffectiveLevel())
assert logger.getEffectiveLevel() == logging.WARNING

logger.debug("This is a debug message")
logger.info("This is an info")
logger.warning("This is a warning")
logger.error("This is an error")

30

[WARNING] __main__: This is a warning
[ERROR] __main__: This is an error


In [ ]:
INVALID_UUID_FOR_TESTING = "00000000-0000-0000-0000-000000000000"

In [ ]:
# Helper context manager for testing

_airt_service_token = None


@contextmanager
def normal_user():
    global _airt_service_token
    if _airt_service_token is None:
        display("_airt_service_token is None, getting a token...")

        username = os.environ[SERVICE_USERNAME]
        password = os.environ[SERVICE_PASSWORD]

        Client.get_token(username=username, password=password)
        _airt_service_token = Client.auth_token

    try:
        os.environ[SERVICE_TOKEN] = _airt_service_token

        yield
    finally:
        del os.environ[SERVICE_TOKEN]

In [ ]:
with normal_user():
    display("*" * len((os.environ[SERVICE_TOKEN])))

'_airt_service_token is None, getting a token...'

'*******************************************************************************************************************************'

In [ ]:
# Helper context manager for testing

_airt_service_super_user_token = None


@contextmanager
def super_user():
    global _airt_service_super_user_token
    if _airt_service_super_user_token is None:
        display("_airt_service_super_user_token is None, getting a token...")

        username = os.environ[SERVICE_SUPER_USER]
        password = os.environ[SERVICE_PASSWORD]

        Client.get_token(username=username, password=password)
        _airt_service_super_user_token = Client.auth_token

    try:
        os.environ[SERVICE_TOKEN] = _airt_service_super_user_token

        yield
    finally:
        del os.environ[SERVICE_TOKEN]

In [ ]:
with super_user():
    display("*" * len((os.environ[SERVICE_TOKEN])))

'_airt_service_super_user_token is None, getting a token...'

'*******************************************************************************************************************************'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def details(
    user: Optional[str] = typer.Option(
        None,
        "--user",
        "-u",
        help="Account user_uuid/username to get details. If not passed, then the currently logged-in details will be returned.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output user uuid only.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Get user details

    Please do not pass the optional 'user' option unless you are a super user. Only a super user can view details for other users.
    """

    from airt.client import User

    df = pd.DataFrame(User.details(user=user), index=[0])[User.USER_COLS]

    return {"df": df}

In [ ]:
def assert_has_help(xs: List[str]):
    result = runner.invoke(app, xs + ["--help"])

    display(result.stdout)
    assert " ".join(xs) in result.stdout

In [ ]:
assert_has_help(["details"])

"Usage: details [OPTIONS]\n\n  Get user details\n\n  Please do not pass the optional 'user' option unless you are a super user.\n  Only a super user can view details for other users.\n\nOptions:\n  -u, --user TEXT                 Account user_uuid/username to get details. If\n                                  not passed, then the currently logged-in\n                                  details will be returned.\n  -f, --format TEXT               Format output and show only the given\n                                  column(s) values.\n  -q, --quiet                     Output user uuid only.\n  --install-completion [bash|zsh|fish|powershell|pwsh]\n                                  Install completion for the specified shell.\n  --show-completion [bash|zsh|fish|powershell|pwsh]\n                                  Show completion for the specified shell, to\n                                  copy it or customize the installation.\n  --help                          Show this message and exit.

In [ ]:
with normal_user():
    # Positive scenario: Normal user getting their details
    format_str = "{'uuid': '{}'}"
    result = runner.invoke(app, ["--format", format_str])
    display(result.stdout)
    assert result.exit_code == 0
    assert len(result.stdout.replace("-", "").replace("\n", "")) == 32

    result = runner.invoke(app, "-q")
    assert result.exit_code == 0
    assert len(result.stdout.replace("-", "").replace("\n", "")) == 32

    user_uuid = result.stdout.replace("\n", "")
    display(user_uuid)

    format_str = "{'username': '{}', 'uuid': '{}'}"
    result = runner.invoke(app, ["--user", user_uuid, "--format", format_str])
    assert result.exit_code == 0

    # Negative scenario: Normal user getting other user's details
    result = runner.invoke(
        app, ["--user", INVALID_UUID_FOR_TESTING, "--format", format_str]
    )
    display(result.stdout)
    assert "Insufficient permission to access other user's data" in result.stdout
    assert result.exit_code == 1

'd12065d3-48cb-4632-a4cc-db11b843399a\n'

'd12065d3-48cb-4632-a4cc-db11b843399a'

"Error: Insufficient permission to access other user's data\n"

In [ ]:
# Helper context manager for testing

_airt_service_super_user_token = None


@contextmanager
def super_user():
    global _airt_service_super_user_token
    if _airt_service_super_user_token is None:
        display("_airt_service_super_user_token is None, getting a token...")

        username = os.environ[SERVICE_SUPER_USER]
        password = os.environ[SERVICE_PASSWORD]

        Client.get_token(username=username, password=password)
        _airt_service_super_user_token = Client.auth_token

    try:
        os.environ[SERVICE_TOKEN] = _airt_service_super_user_token

        yield
    finally:
        del os.environ[SERVICE_TOKEN]

In [ ]:
with super_user():
    # Positive scenario: Super user getting their details
    format_str = "{'username': '{}', 'uuid': '{}'}"
    result = runner.invoke(app, ["--format", format_str])
    assert result.exit_code == 0

    format_str = "{'username': '{}', 'uuid': '{}'}"
    result = runner.invoke(app, "-q")
    assert result.exit_code == 0

    user_uuid = result.stdout.replace("\n", "")
    format_str = "{'username': '{}', 'uuid': '{}'}"
    result = runner.invoke(app, ["--user", user_uuid, "--format", format_str])
    assert result.exit_code == 0

    # Positive scenario: Super user accessing others user's details
    other_user = User.ls()[0]
    result = runner.invoke(app, ["--user", other_user.uuid, "-q"])
    display(result.stdout)
    result = runner.invoke(app, ["--user", other_user.username, "-q"])
    display(result.stdout)

    # Negative scenario: Super user accessing invalid user's details
    result = runner.invoke(
        app, ["--user", INVALID_UUID_FOR_TESTING, "--format", format_str]
    )
    display(result.stdout)
    assert "user uuid is incorrect" in result.stdout
    assert result.exit_code == 1

    invalid_user_name = "invalid_user_name"
    result = runner.invoke(app, ["--user", invalid_user_name, "--format", format_str])
    display(result.stdout)
    assert "Incorrect username" in result.stdout
    assert result.exit_code == 1

'_airt_service_super_user_token is None, getting a token...'

'd12065d3-48cb-4632-a4cc-db11b843399a\n'

'd12065d3-48cb-4632-a4cc-db11b843399a\n'

'Error: The user uuid is incorrect. Please try again.\n'

'Error: Incorrect username. Please try again.\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_totp()
@helper.requires_auth_token
def create(
    username: str = typer.Option(
        ...,
        "--username",
        "-un",
        help="The new user's username. The username must be unique or an exception will be thrown.",
    ),
    first_name: str = typer.Option(
        ...,
        "--first_name",
        "-fn",
        help="The new user's first name.",
    ),
    last_name: str = typer.Option(
        ...,
        "--last_name",
        "-ln",
        help="The new user's last name.",
    ),
    email: str = typer.Option(
        ...,
        "--email",
        "-e",
        help="The new user's email. The email must be unique or an exception will be thrown.",
    ),
    password: str = typer.Option(
        ...,
        "--password",
        "-p",
        help="The new user's password.",
    ),
    subscription_type: str = typer.Option(
        ...,
        "--subscription_type",
        "-st",
        help="User subscription type. Currently, the API supports only the following subscription types **small**, **medium** and **large**.",
    ),
    super_user: bool = typer.Option(
        False,
        "--super_user",
        "-su",
        help="If set to **True**, then the new user will have super user privilages. If **None**, then the default value "
        "**False** will be used to create a non-super user.",
    ),
    phone_number: Optional[str] = typer.Option(
        None,
        "--phone_number",
        "-ph",
        help="Phone number to be added to the user account. The phone number should follow the pattern of the country "
        "code followed by your phone number. For example, 440123456789, +440123456789, 00440123456789, +44 0123456789,"
        "and (+44) 012 345 6789 are all valid formats for registering a UK phone number.",
    ),
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app. Please pass this optional argument only if you have activated the MFA for your account.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output user uuid only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Create a new user in the server."""

    from airt.client import User

    df = User.create(
        username=username,
        first_name=first_name,
        last_name=last_name,
        email=email,
        subscription_type=subscription_type,
        super_user=super_user,
        password=password,
        phone_number=phone_number,
        otp=otp,
    )

    df["created"] = helper.humanize_date(df["created"])

    return {"df": df}

In [ ]:
# tests for user create. Without -q

_user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
_email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"

format_str = "{'username': '{}', 'uuid': '{}', 'phone_number': '{}'}"
cmd = [
    "create",
    "-un",
    _user_name,
    "-fn",
    "random_first_name",
    "-ln",
    "random_last_name",
    "-e",
    _email,
    "-p",
    "random_password",
    "-st",
    "small",
    "-ph",
    "+44123456789",
    "-su",
    "--format",
    format_str,
]

_user_name_q = f"random_user_{randrange(10000)}_{randrange(10000)}"
_email_q = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"


cmd_with_otp = [
    "create",
    "-un",
    _user_name_q,
    "-fn",
    "random_first_name",
    "-ln",
    "random_last_name",
    "-e",
    _email_q,
    "-p",
    "random_password",
    "-st",
    "small",
    "-su",
    "--otp",
    "123456",
    "--format",
    format_str,
]

with super_user():
    result = runner.invoke(app, cmd)

    display(result.stdout)
    assert result.exit_code == 0
    assert _user_name in str(result.stdout)
    assert "44123456789" in str(result.stdout)

    # Trying to create new user with existing email and username
    result = runner.invoke(app, cmd)
    display(result.stdout)
    assert result.exit_code == 1

    # Negative Scenario. Non-MFA user passing otp
    random_otp = 123456
    result = runner.invoke(app, cmd_with_otp)
    display(result.stdout)
    assert result.exit_code == 1

'username               uuid                                    phone_number\nrandom_user_1772_2579  a3872746-c793-4c4a-864e-7d05a7d1e6ed     44123456789\n'

'Error: The requested username or email already exists. Try another.\n'

'Error: MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.\n'

In [ ]:
# tests for user create. With -q

_user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
_email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"


cmd = [
    "create",
    "-un",
    _user_name,
    "-fn",
    "random_first_name",
    "-ln",
    "random_last_name",
    "-e",
    _email,
    "-p",
    "random_password",
    "-st",
    "small",
    "-su",
    "-q",
]

with super_user():
    result = runner.invoke(app, cmd)
    user_uuid = result.stdout[:-1]

    display(result.stdout)
    assert result.exit_code == 0
    assert len(result.stdout.replace("-", "").replace("\n", "")) == 32

'393cc29c-2671-4fcd-941c-d0cb4c6f864b\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_auth_token
def ls(
    offset: int = typer.Option(
        0,
        "--offset",
        "-o",
        help="The number of users to offset at the beginning. If **None**, then the default value **0** will be used.",
    ),
    limit: int = typer.Option(
        100,
        "--limit",
        "-l",
        help="The maximum number of users to return from the server. If None, then the default value 100 will be used.",
    ),
    disabled: bool = typer.Option(
        False,
        "--disabled",
        help="If set to **True**, then only the deleted users will be returned. Else, the default value **False** will "
        "be used to return only the list of active users.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output only user uuids separated by space",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Return the list of users available in the server."""

    from airt.client import User

    ux = User.ls(offset=offset, limit=limit, disabled=disabled)

    df = User.as_df(ux)

    df["created"] = helper.humanize_date(df["created"])

    return {"df": df}

In [ ]:
assert_has_help(["ls"])

'Usage: root ls [OPTIONS]\n\n  Return the list of users available in the server.\n\nOptions:\n  -o, --offset INTEGER  The number of users to offset at the beginning. If\n                        **None**, then the default value **0** will be used.\n                        [default: 0]\n  -l, --limit INTEGER   The maximum number of users to return from the server.\n                        If None, then the default value 100 will be used.\n                        [default: 100]\n  --disabled            If set to **True**, then only the deleted users will be\n                        returned. Else, the default value **False** will be used\n                        to return only the list of active users.\n  -f, --format TEXT     Format output and show only the given column(s) values.\n  -q, --quiet           Output only user uuids separated by space\n  -d, --debug           Set logger level to DEBUG and output everything.\n  --help                Show this message and exit.\n'

In [ ]:
# Tests for ls
# Testing positive scenario. Saving the token in env variable


def get_uuids_from_result(result) -> List[str]:
    return [uuid for uuid in result.stdout[:-1].split("\n")]


with super_user():

    # Without quiet
    format_str = "{'username': '{}', 'uuid': '{}'}"
    result = runner.invoke(app, ["ls", "--format", format_str])
    assert "username" in result.stdout
    assert result.exit_code == 0

    # With quiet
    result = runner.invoke(app, ["ls", "-q"])
    display(result.stdout)

    assert result.exit_code == 0
    ids = get_uuids_from_result(result)
    display(f"{ids=}")

'd12065d3-48cb-4632-a4cc-db11b843399a\ne53369c4-f5ea-478d-a078-147c0df00093\n7f14fe1b-3c44-4bad-b1b8-e0da5a3429e8\n68eb0ff7-9004-4ee6-baa2-4e6bfc62b68a\n95c0ff28-4e0a-4be3-88ae-583bf1027ae7\n8b872c68-5a47-4d43-b565-c2215b5c144c\n8b8fee72-a02a-4b23-8eb4-6d438d67d5a8\n4130a386-a377-4ea1-9309-c62442a66637\na7b89f87-ceac-43ed-84c6-fc96ec0a52a1\n1b6b11a9-a5f3-4ba6-a244-e3ec8e917798\ne4c31874-a867-4fc6-8407-531fc4711c39\n45b58d1a-9716-4e39-8a2a-4fa34e64c681\nc2b6fa37-f439-48e9-9f60-d801ebd24920\nf0853c45-5b0e-4ee9-87bc-c92235a1651d\n90749653-705c-48ff-bfda-82eb3b7d840e\ncbe73708-4ebc-4a15-9920-c5bfe07dc4a6\n5f46973b-023d-4400-b2b3-4096aac0029a\n452a0458-c450-4eeb-a6e2-937ae6ad8e6a\nf04bd56a-2226-4a0d-ace3-fe9497736fb5\n274d569b-c202-45a9-8a24-13d9dc525d84\n6ef10bbe-fd24-4be2-bc21-7cf559979abd\n016a5fe5-db60-4ef6-bfa4-f779ac4e01dc\n07f88176-5fe1-4847-85d5-51e93aebff20\naf7e2314-6c56-4b4e-a90c-cfebdae0ea52\n2c4208d7-fdf8-48a9-bf54-ac72afaf6d3d\nd02dd242-31d4-40e4-99e3-2a49be006093\na3872746-c7

"ids=['d12065d3-48cb-4632-a4cc-db11b843399a', 'e53369c4-f5ea-478d-a078-147c0df00093', '7f14fe1b-3c44-4bad-b1b8-e0da5a3429e8', '68eb0ff7-9004-4ee6-baa2-4e6bfc62b68a', '95c0ff28-4e0a-4be3-88ae-583bf1027ae7', '8b872c68-5a47-4d43-b565-c2215b5c144c', '8b8fee72-a02a-4b23-8eb4-6d438d67d5a8', '4130a386-a377-4ea1-9309-c62442a66637', 'a7b89f87-ceac-43ed-84c6-fc96ec0a52a1', '1b6b11a9-a5f3-4ba6-a244-e3ec8e917798', 'e4c31874-a867-4fc6-8407-531fc4711c39', '45b58d1a-9716-4e39-8a2a-4fa34e64c681', 'c2b6fa37-f439-48e9-9f60-d801ebd24920', 'f0853c45-5b0e-4ee9-87bc-c92235a1651d', '90749653-705c-48ff-bfda-82eb3b7d840e', 'cbe73708-4ebc-4a15-9920-c5bfe07dc4a6', '5f46973b-023d-4400-b2b3-4096aac0029a', '452a0458-c450-4eeb-a6e2-937ae6ad8e6a', 'f04bd56a-2226-4a0d-ace3-fe9497736fb5', '274d569b-c202-45a9-8a24-13d9dc525d84', '6ef10bbe-fd24-4be2-bc21-7cf559979abd', '016a5fe5-db60-4ef6-bfa4-f779ac4e01dc', '07f88176-5fe1-4847-85d5-51e93aebff20', 'af7e2314-6c56-4b4e-a90c-cfebdae0ea52', '2c4208d7-fdf8-48a9-bf54-ac72afaf6

In [ ]:
# tests for user ls

_user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
_email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"

cmd = [
    "create",
    "-un",
    _user_name,
    "-fn",
    "random_first_name",
    "-ln",
    "random_last_name",
    "-e",
    _email,
    "-p",
    "random_password",
    "-st",
    "small",
    "-su",
    "-q",
]

with super_user():
    # creating a new user
    result = runner.invoke(app, cmd)
    created_user_uuid = result.stdout[:-1]

    display(f"{created_user_uuid=}\n")

    # checking the user id is not there in the ls when --disabled is passed
    result = runner.invoke(app, ["ls", "--disabled", "-q"])
    display(result.stdout)
    assert result.exit_code == 0
    assert f"{created_user_uuid}" not in str(result.stdout)

"created_user_uuid='6d007a52-c15e-404c-945a-9396d38459e2'\n"

'9b4c8afb-8c8f-4e26-b3a3-094dc0a29cd3\na2b14123-0729-4194-a22d-8538facb7da1\n003a19ea-a7ce-4f17-b959-8f4e7b05b261\nc14e1d0c-cd19-4699-a9c6-cfe04b9b1df7\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_totp()
@helper.requires_auth_token
def disable(
    users: List[str] = typer.Argument(
        ...,
        help="user_uuid/username to disabled.  To disable multiple users, please pass the uuids/names separated by space.",
    ),
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app. Please pass this optional argument only if you have activated the MFA for your account.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output user uuid only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Disable a user in the server."""

    from airt.client import User

    users = [user for user in users]
    #     formated_users = helper.separate_integers_and_strings(users)

    df = User.disable(user=users, otp=otp)  # type: ignore
    df["created"] = helper.humanize_date(df["created"])

    return {"df": df}

In [ ]:
assert_has_help(["disable"])

'Usage: root disable [OPTIONS] USERS...\n\n  Disable a user in the server.\n\nArguments:\n  USERS...  user_uuid/username to disabled.  To disable multiple users, please\n            pass the uuids/names separated by space.  [required]\n\nOptions:\n  --otp TEXT         Dynamically generated six-digit verification code from the\n                     authenticator app. Please pass this optional argument only\n                     if you have activated the MFA for your account.\n  -f, --format TEXT  Format output and show only the given column(s) values.\n  -q, --quiet        Output user uuid only.\n  -d, --debug        Set logger level to DEBUG and output everything.\n  --help             Show this message and exit.\n'

In [ ]:
# tests for user disable

# creating a new user
_user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
_email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"

cmd = [
    "create",
    "-un",
    _user_name,
    "-fn",
    "random_first_name",
    "-ln",
    "random_last_name",
    "-e",
    _email,
    "-p",
    "random_password",
    "-st",
    "small",
    "-su",
    "-q",
]

with super_user():

    # Creating a new user with super user credentials
    result = runner.invoke(app, cmd)
    created_user_uuid = result.stdout[:-1]

    assert result.exit_code == 0

    # disabling the newly created user
    created_user_name = runner.invoke(
        app, ["details", "--user", created_user_uuid, "--format", "{'username': '{}'}"]
    )
    result = runner.invoke(app, ["disable", created_user_name.stdout[:-1], "-q"])
    disabled_user_uuid = result.stdout[:-1]

    display(f"{disabled_user_uuid=}")

    # list disabled users only
    result = runner.invoke(app, ["ls", "--disabled", "-q"])
    all_disabled_users = list(get_uuids_from_result(result))

    display(f"{all_disabled_users=}")

    assert result.exit_code == 0
    assert disabled_user_uuid in all_disabled_users

    # disabling already disabled user
    result = runner.invoke(app, ["disable", created_user_uuid])

    display(result.stdout)

    assert result.exit_code == 1

    # Negative Scenario. Non-MFA user passing otp
    random_otp = 123456
    result = runner.invoke(app, ["disable", created_user_uuid, "--otp", random_otp])
    display(result.stdout)
    assert result.exit_code == 1

"disabled_user_uuid='c8e24693-2f7e-4e4a-bad0-0f2888d7216f'"

"all_disabled_users=['9b4c8afb-8c8f-4e26-b3a3-094dc0a29cd3', 'a2b14123-0729-4194-a22d-8538facb7da1', '003a19ea-a7ce-4f17-b959-8f4e7b05b261', 'c14e1d0c-cd19-4699-a9c6-cfe04b9b1df7', 'c8e24693-2f7e-4e4a-bad0-0f2888d7216f']"

'Error: The user has already been disabled.\n'

'Error: MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.\n'

In [ ]:
# creating a new user
_user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
_email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"

format_str = "{'username': '{}', 'uuid': '{}'}"
cmd = [
    "create",
    "-un",
    _user_name,
    "-fn",
    "random_first_name",
    "-ln",
    "random_last_name",
    "-e",
    _email,
    "-p",
    "random_password",
    "-st",
    "small",
    "-su",
    "--format",
    format_str,
]

with super_user():

    # Creating a new user with super user credentials
    result = runner.invoke(app, cmd)
    created_user_uuid = result.stdout[:-1]
    display(result.stdout)

    assert result.exit_code == 0

'username               uuid\nrandom_user_7131_6824  989d1a2f-ea59-4171-b7b4-dc4dcbd97f8d\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_totp()
@helper.requires_auth_token
def enable(
    users: List[str] = typer.Argument(
        ...,
        help="user_uuid/username to enable. To enable multiple users, please pass the uuids/names separated by space.",
    ),
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app. Please pass this optional argument only if you have activated the MFA for your account.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output user uuid only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Enable a disabled user in the server."""

    from airt.client import User

    users = [user for user in users]
    #     formated_users = helper.separate_integers_and_strings(users)

    df = User.enable(user=users, otp=otp)  # type: ignore
    df["created"] = helper.humanize_date(df["created"])

    return {"df": df}

In [ ]:
assert_has_help(["enable"])

'Usage: root enable [OPTIONS] USERS...\n\n  Enable a disabled user in the server.\n\nArguments:\n  USERS...  user_uuid/username to enable. To enable multiple users, please pass\n            the uuids/names separated by space.  [required]\n\nOptions:\n  --otp TEXT         Dynamically generated six-digit verification code from the\n                     authenticator app. Please pass this optional argument only\n                     if you have activated the MFA for your account.\n  -f, --format TEXT  Format output and show only the given column(s) values.\n  -q, --quiet        Output user uuid only.\n  -d, --debug        Set logger level to DEBUG and output everything.\n  --help             Show this message and exit.\n'

In [ ]:
# tests for user enable

# creating a new user
_user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
_email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"

cmd = [
    "create",
    "-un",
    _user_name,
    "-fn",
    "random_first_name",
    "-ln",
    "random_last_name",
    "-e",
    _email,
    "-p",
    "random_password",
    "-st",
    "small",
    "-su",
    "-q",
]

with super_user():

    # Creating a new user
    result = runner.invoke(app, cmd)
    created_user_uuid = result.stdout[:-1]

    assert result.exit_code == 0

    # disabling the newly created user
    result = runner.invoke(app, ["disable", created_user_uuid, "-q"])
    disabled_user_uuid = result.stdout[:-1]
    display(result.stdout)
    assert result.exit_code == 0

    # enabling the newly created user
    created_user_name = runner.invoke(
        app, ["details", "--user", created_user_uuid, "--format", "{'username': '{}'}"]
    )
    result = runner.invoke(app, ["enable", created_user_name.stdout[:-1], "-q"])
    enabled_user_id = result.stdout[:-1]

    display(f"{enabled_user_id=}")

    # list enabled users only
    result = runner.invoke(app, ["ls", "-q", "-l", 500])
    all_enabled_users = list(get_uuids_from_result(result))

    display(f"{all_enabled_users=}")

    assert result.exit_code == 0
    assert enabled_user_id in all_enabled_users

    # enabling already enabled user
    result = runner.invoke(app, ["enable", created_user_uuid])

    display(result.stdout)

    assert result.exit_code == 1

    # Negative Scenario. Non-MFA user passing otp
    random_otp = 123456
    result = runner.invoke(app, ["enable", created_user_uuid, "--otp", random_otp])
    display(result.stdout)
    assert result.exit_code == 1

'4984a628-ef22-453a-9433-9f11e7d054b0\n'

"enabled_user_id='4984a628-ef22-453a-9433-9f11e7d054b0'"

"all_enabled_users=['d12065d3-48cb-4632-a4cc-db11b843399a', 'e53369c4-f5ea-478d-a078-147c0df00093', '7f14fe1b-3c44-4bad-b1b8-e0da5a3429e8', '68eb0ff7-9004-4ee6-baa2-4e6bfc62b68a', '95c0ff28-4e0a-4be3-88ae-583bf1027ae7', '8b872c68-5a47-4d43-b565-c2215b5c144c', '8b8fee72-a02a-4b23-8eb4-6d438d67d5a8', '4130a386-a377-4ea1-9309-c62442a66637', 'a7b89f87-ceac-43ed-84c6-fc96ec0a52a1', '1b6b11a9-a5f3-4ba6-a244-e3ec8e917798', 'e4c31874-a867-4fc6-8407-531fc4711c39', '45b58d1a-9716-4e39-8a2a-4fa34e64c681', 'c2b6fa37-f439-48e9-9f60-d801ebd24920', 'f0853c45-5b0e-4ee9-87bc-c92235a1651d', '90749653-705c-48ff-bfda-82eb3b7d840e', 'cbe73708-4ebc-4a15-9920-c5bfe07dc4a6', '5f46973b-023d-4400-b2b3-4096aac0029a', '452a0458-c450-4eeb-a6e2-937ae6ad8e6a', 'f04bd56a-2226-4a0d-ace3-fe9497736fb5', '274d569b-c202-45a9-8a24-13d9dc525d84', '6ef10bbe-fd24-4be2-bc21-7cf559979abd', '016a5fe5-db60-4ef6-bfa4-f779ac4e01dc', '07f88176-5fe1-4847-85d5-51e93aebff20', 'af7e2314-6c56-4b4e-a90c-cfebdae0ea52', '2c4208d7-fdf8-48a9-

'Error: The user has already been enabled.\n'

'Error: MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.\n'

In [ ]:
# tests for enabling muliple users

# creating a new user
def get_cmd_to_create_user():
    random_user = randrange(10000)
    cmd = [
        "create",
        "-un",
        f"random_user_{random_user}",
        "-fn",
        "random_first_name",
        "-ln",
        "random_last_name",
        "-e",
        f"random_user_{random_user}@email.com",
        "-p",
        "random_password",
        "-st",
        "small",
        "-su",
        "-q",
    ]
    return cmd


with super_user():

    # Creating a new user
    result = runner.invoke(app, get_cmd_to_create_user())
    display(result.stdout)
    user_1 = result.stdout[:-1]

    assert result.exit_code == 0

    # Creating a new user
    result = runner.invoke(app, get_cmd_to_create_user())
    display(result.stdout)
    user_2 = result.stdout[:-1]

    assert result.exit_code == 0

    # disabling the newly created user
    result = runner.invoke(app, ["disable", user_1, user_2, "-q"])
    disabled_user_uuid = result.stdout[:-1]
    disabled_user_1_uuid = disabled_user_uuid.split("\n")[0]
    disabled_user_2_uuid = disabled_user_uuid.split("\n")[1]

    # enabling multiplee users
    result = runner.invoke(app, ["enable", disabled_user_1_uuid, disabled_user_2_uuid])
    assert disabled_user_1_uuid in result.stdout
    assert disabled_user_2_uuid in result.stdout

'44de0383-288a-4595-9331-ee5ad27dc0d2\n'

'efb4f4ab-eee5-4c57-8e20-e0368bea0415\n'

In [ ]:
# creating a new user
_user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
_email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"


cmd = [
    "create",
    "-un",
    _user_name,
    "-fn",
    "random_first_name",
    "-ln",
    "random_last_name",
    "-e",
    _email,
    "-p",
    "random_password",
    "-st",
    "small",
    "-su",
    "-q",
]

with super_user():

    # Creating a new user with super user credentials
    result = runner.invoke(app, cmd)
    created_user_uuid = result.stdout[:-1]

    assert result.exit_code == 0

    # disabling the newly created user
    result = runner.invoke(app, ["disable", created_user_uuid, "-q"])
    disabled_user_uuid = result.stdout[:-1]

    assert result.exit_code == 0

    # enabling the newly created user
    format_str = "{'username': '{}', 'uuid': '{}'}"
    result = runner.invoke(
        app, ["enable", created_user_uuid, "-q", "--format", format_str]
    )
    display(result.stdout)

'username               uuid\nrandom_user_2376_4264  3be0acb4-fce8-4ae3-a004-ae3101617fe0\n'

In [ ]:
# creating a new user
_user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
_email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"


cmd = [
    "create",
    "-un",
    _user_name,
    "-fn",
    "random_first_name",
    "-ln",
    "random_last_name",
    "-e",
    _email,
    "-p",
    "random_password",
    "-st",
    "small",
    "-su",
    "-q",
]

with super_user():

    # Creating a new user with super user credentials
    result = runner.invoke(app, cmd)
    created_user_uuid = result.stdout[:-1]

    assert result.exit_code == 0

    # disabling the newly created user
    result = runner.invoke(app, ["disable", created_user_uuid, "-q"])
    disabled_user_uuid = result.stdout[:-1]

    assert result.exit_code == 0

    # enabling the newly created user
    format_str = "{'username': '{}', 'uuid': '{}'}"
    result = runner.invoke(
        app, ["enable", created_user_uuid, "-q", "--format", format_str]
    )
    display(result.stdout)

'username               uuid\nrandom_user_1820_2809  3685e5d0-9e51-4b9d-afb5-09a5e5293bf4\n'

In [ ]:
# | exporti


@app.command()
@helper.display_formated_table
@helper.requires_totp()
@helper.requires_auth_token
def update(
    user: Optional[str] = typer.Option(
        None,
        "--user",
        help="Account user_uuid/username to update. If not passed, then the default value None will be used to update the currently logged-in user details.",
    ),
    username: Optional[str] = typer.Option(
        None,
        "--username",
        "-un",
        help="New username for the user.",
    ),
    first_name: Optional[str] = typer.Option(
        None,
        "--first_name",
        "-fn",
        help="New first name for the user.",
    ),
    last_name: Optional[str] = typer.Option(
        None,
        "--last_name",
        "-ln",
        help="New last name for the user.",
    ),
    email: Optional[str] = typer.Option(
        None,
        "--email",
        "-e",
        help="New email for the user.",
    ),
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app. Please pass this optional argument only if you have activated the MFA for your account.",
    ),
    format: Optional[str] = typer.Option(
        None,
        "--format",
        "-f",
        help="Format output and show only the given column(s) values.",
    ),
    quiet: bool = typer.Option(
        False,
        "--quiet",
        "-q",
        help="Output user uuid only.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
) -> Dict["str", Union[pd.DataFrame, str]]:
    """Update existing user details in the server.

    Please do not pass the optional user option unless you are a super user. Only a
    super user can update details for other users.

    """

    from airt.client import User

    df = User.update(
        user=user,
        username=username,
        first_name=first_name,
        last_name=last_name,
        email=email,
        otp=otp,
    )

    df["created"] = helper.humanize_date(df["created"])

    return {"df": df}

In [ ]:
assert_has_help(["update"])

'Usage: root update [OPTIONS]\n\n  Update existing user details in the server.\n\n  Please do not pass the optional user option unless you are a super user. Only\n  a super user can update details for other users.\n\nOptions:\n  --user TEXT             Account user_uuid/username to update. If not passed,\n                          then the default value None will be used to update the\n                          currently logged-in user details.\n  -un, --username TEXT    New username for the user.\n  -fn, --first_name TEXT  New first name for the user.\n  -ln, --last_name TEXT   New last name for the user.\n  -e, --email TEXT        New email for the user.\n  --otp TEXT              Dynamically generated six-digit verification code from\n                          the authenticator app. Please pass this optional\n                          argument only if you have activated the MFA for your\n                          account.\n  -f, --format TEXT       Format output and show only the gi

In [ ]:
# tests for user update

# Testing positive scenario. Updating the user details as super user

_user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
_email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"


cmd = [
    "create",
    "-un",
    _user_name,
    "-fn",
    "random_first_name",
    "-ln",
    "random_last_name",
    "-e",
    _email,
    "-p",
    "random_password",
    "-st",
    "small",
    "-su",
    "-q",
]

with super_user():

    # Creating a new user with super user credentials
    result = runner.invoke(app, cmd)
    created_user_uuid = result.stdout[:-1]
    assert result.exit_code == 0

    # updating the user details as super user
    format_str = "{'first_name': '{}', 'uuid': '{}'}"
    result = runner.invoke(
        app,
        [
            "update",
            "--user",
            created_user_uuid,
            "--first_name",
            "new_first_name",
            "--format",
            format_str,
        ],
    )

    display(result.stdout)
    assert result.exit_code == 0
    assert "new_first_name" in result.stdout

    # Negative Scenario. Non-MFA user passing otp
    random_otp = 123456
    result = runner.invoke(
        app,
        [
            "update",
            "--user",
            created_user_uuid,
            "--first_name",
            "new_first_name",
            "--format",
            format_str,
            "--otp",
            random_otp,
        ],
    )
    display(result.stdout)
    assert "MFA is not activated for the account." in result.stdout, result.stdout
    assert result.exit_code == 1

'first_name      uuid\nnew_first_name  42a7aad7-0d01-4e40-9db4-a8fc5d9a3ee5\n'

'Error: MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.\n'

In [ ]:
# tests for user update

# Testing positive scenario: Normal user updating their details

_email = f"random_user_{randrange(10000)}_{randrange(10000)}@email.com"

_new_user_id = None
_new_user_name = f"random_user_{randrange(10000)}_{randrange(10000)}"
_new_user_password = "random_password"

cmd = [
    "create",
    "-un",
    _new_user_name,
    "-fn",
    "random_first_name",
    "-ln",
    "random_last_name",
    "-e",
    _email,
    "-p",
    _new_user_password,
    "-st",
    "small",
    "-su",
    "-q",
]

with super_user():
    global _new_user_id

    # Creating a new user with super user credentials
    result = runner.invoke(app, cmd)
    _new_user_id = result.stdout[:-1]

    assert result.exit_code == 0

# Testing positive scenario: Normal user updating their details
try:

    Client.get_token(username=_new_user_name, password=_new_user_password)

    os.environ[SERVICE_TOKEN] = Client.auth_token

    # Updating the user details
    _updated_first_name = "new_first_name"
    _updated_last_name = "new_last_name"

    result = runner.invoke(
        app,
        [
            "update",
            "--first_name",
            _updated_first_name,
            "-ln",
            _updated_last_name,
        ],
    )

    display(result.stdout)
    assert result.exit_code == 0
    assert _updated_first_name in result.stdout
    assert _updated_last_name in result.stdout

    # logging in with the new password
    Client.get_token(username=_new_user_name, password=_new_user_password)
    display(f"Masked token = { '*' * (len(Client.auth_token))}")
    assert len(Client.auth_token) >= 127

    # Testing negative scenario: Updating other user details with normal user credentials
    result = runner.invoke(
        app,
        [
            "update",
            "--user",
            INVALID_UUID_FOR_TESTING,
            "--first_name",
            _updated_first_name,
            "-ln",
            _updated_last_name,
        ],
    )
    assert result.exit_code == 1

finally:
    del os.environ[SERVICE_TOKEN]

'uuid                                  username               email                            super_user    is_mfa_active    disabled    created    subscription_type    first_name      last_name      phone_number    is_phone_number_verified\n88341c0c-eba0-4019-845b-de698a5b381a  random_user_4748_8390  random_user_4706_7056@email.com  True          False            False       now        small                new_first_name  new_last_name  <none>          False\n'

'Masked token = *************************************************************************************************************************************************'

In [ ]:
# | exporti


@app.command("register-phone-number")
@helper.requires_totp()
@helper.requires_auth_token
def register_phone_number(
    phone_number: Optional[str] = typer.Option(
        None,
        "--phone-number",
        "-p",
        help="""Phone number to register. The phone number should follow the pattern of the 
            country code followed by your phone number. For example, **440123456789, +440123456789,
            00440123456789, +44 0123456789, and (+44) 012 345 6789** are all valid formats for registering a UK phone number.
            If the phone number is not passed in the arguments, then the OTP will be sent to the phone 
            number that was already registered to the user's account.""",
    ),
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app. Please pass this optional argument only if you have activated the MFA for your account.",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
):
    """Register and validate a phone number

    This is an interactive command, one called it will send an OTP via SMS to the phone number. Please enter the OTP you have received
    in the interactive prompt to complete the phone number registration process.

    After ten invalid OTP attempts, you have to call this command again to register the phone number.
    """

    from airt.client import User

    user = User.register_phone_number(phone_number=phone_number, otp=otp)

    while True:
        try:
            sms_otp = typer.prompt(
                f"We have sent a One-Time Password (OTP) to the phone number {user['phone_number']}. Please enter here"
            )
            typer.echo("\n")
            response = User.validate_phone_number(otp=sms_otp)
            typer.echo(
                f"The phone number {user['phone_number']} is successfully registered. We will send the OTP via SMS to this registered phone number when requested."
            )
            break
        except ValueError as e:
            typer.echo(e)
            if ("Too many failed attempts" in str(e)) or (
                "OTP entered is expired" in str(e)
            ):
                break

In [ ]:
result = runner.invoke(app, ["register-phone-number", "--help"])

display(result.stdout)
assert "register-phone-number" in result.stdout

"Usage: root register-phone-number [OPTIONS]\n\n  Register and validate a phone number\n\n  This is an interactive command, one called it will send an OTP via SMS to the\n  phone number. Please enter the OTP you have received in the interactive prompt\n  to complete the phone number registration process.\n\n  After ten invalid OTP attempts, you have to call this command again to\n  register the phone number.\n\nOptions:\n  -p, --phone-number TEXT  Phone number to register. The phone number should\n                           follow the pattern of the  country code followed by\n                           your phone number. For example, **440123456789,\n                           +440123456789, 00440123456789, +44 0123456789, and\n                           (+44) 012 345 6789** are all valid formats for\n                           registering a UK phone number. If the phone number is\n                           not passed in the arguments, then the OTP will be\n                           

In [ ]:
# | exporti


@app.command("reset-password")
def reset_password(
    username: Optional[str] = typer.Option(
        None, "--username", "-u", help="Account username to reset the password"
    ),
    new_password: Optional[str] = typer.Option(
        None, "--new-password", "-np", help="New password to set for the account"
    ),
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app",
    ),
    debug: bool = typer.Option(
        False,
        "--debug",
        "-d",
        help="Set logger level to DEBUG and output everything.",
    ),
):
    """Reset the account password

    We currently support two types of OTPs to reset the password for your account and you don't have to be logged in to call this command

    \n\nThe command switches to interactive mode unless all arguments are passed. The interactive mode will prompt you for the missing details and ask you to choose a recovery option to reset your password. Currently, we only support resetting the password either using a TOTP or SMS OTP.
    \n\nIf you have already activated the MFA for your account, then you can either enter the dynamically generated six-digit verification code from the authenticator app (TOTP) or request an OTP via SMS to your registered phone number.
    \n\nIf the MFA is not activated already, then you can only request the OTP via SMS to your registered phone number.
    \n\nAfter selecting an option, please follow the on-screen instructions to reset your password. In case, you don't have MFA enabled or don't have access to your registered phone number, please contact your administrator.
    """

    from airt.client import User

    if username is None:
        username = typer.prompt("Please enter your username")

    if new_password is None:
        new_password = typer.prompt(
            "Please enter your new password", hide_input=True, confirmation_prompt=True
        )

    if otp is not None:
        try:
            status = User.reset_password(
                username=username, new_password=new_password, otp=otp
            )
            typer.echo(f"\n{status}")
            typer.echo(
                f"\nPlease don't forget to set the updated password in the `{SERVICE_PASSWORD}` environment variable"
            )
        except ValueError as e:
            typer.echo(e)
            raise typer.Exit(code=1)
    else:
        typer.echo("\nPlease choose an option to reset your password\n\n")

        while True:
            typer.echo(
                "[1] Reset password using the dynamically generated six-digit verification code from the authenticator application\n"
            )
            typer.echo(
                "[2] Reset password by requesting the OTP via SMS to the registered phone number\n"
            )
            typer.echo(
                "If you cannot access the authenticator application or your registered phone number, please contact your administrator.\n"
            )

            recovery_option = typer.prompt("Password reset option")

            if recovery_option in ["1", "2"]:
                break
            typer.echo("Please enter a valid password reset option")

        if recovery_option == "1":
            try:
                totp = typer.prompt(
                    "Please enter the OTP displayed in the authenticator application"
                )
                status = User.reset_password(
                    username=username, new_password=new_password, otp=totp
                )
            except ValueError as e:
                typer.echo(e)

        elif recovery_option == "2":
            try:
                sms_status = User.send_sms_otp(
                    username=username, message_template_name="reset_password"
                )
                typer.echo(f"\n{sms_status}\n")
                sms_otp = typer.prompt(
                    f"Please enter the One-Time Password (OTP) you received on your registered phone number"
                )
                status = User.reset_password(
                    username=username, new_password=new_password, otp=sms_otp
                )
            except ValueError as e:
                typer.echo(e)

        else:
            raise typer.Exit(code=1)

        typer.echo(f"\n{status}")

        if os.environ.get(SERVICE_PASSWORD) is not None:

            typer.echo(
                f"\nPlease don't forget to set the updated password in the `{SERVICE_PASSWORD}` environment variable"
            )

In [ ]:
result = runner.invoke(app, ["reset-password", "--help"])

display(result.stdout)
assert "reset-password" in result.stdout

"Usage: root reset-password [OPTIONS]\n\n  Reset the account password\n\n      We currently support two types of OTPs to reset the password for your\n      account and you don't have to be logged in to call this command\n\n\n\n  The command switches to interactive mode unless all arguments are passed. The\n  interactive mode will prompt you for the missing details and ask you to choose\n  a recovery option to reset your password. Currently, we only support resetting\n  the password either using a TOTP or SMS OTP.\n\n  If you have already activated the MFA for your account, then you can either\n  enter the dynamically generated six-digit verification code from the\n  authenticator app (TOTP) or request an OTP via SMS to your registered phone\n  number.\n\n  If the MFA is not activated already, then you can only request the OTP via SMS\n  to your registered phone number.\n\n  After selecting an option, please follow the on-screen instructions to reset\n  your password. In case, you don't

In [ ]:
# Tests for reset_password: passing invalid totp

cmd = [
    "reset-password",
    "-u",
    "invalid_username",
    "-np",
    "invalid_password",
    "--otp",
    "123456",
]

result = runner.invoke(app, cmd)
assert "Something went wrong" in str(result.stdout), str(result.stdout)
display(result.stdout)

'Something went wrong. The username or OTP you entered is incorrect. Please try again or contact your administrator.\n'

In [ ]:
# | exporti

mfa_app = typer.Typer(
    help="Commands for enabling and disabling Multi-Factor Authentication (MFA)."
)

In [ ]:
# | exporti


@mfa_app.command()  # type: ignore
@helper.requires_totp()
@helper.requires_auth_token
def enable(
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app. Please pass this optional argument only if you have activated the MFA for your account.",
    ),
) -> None:
    """Enable Multi-Factor Authentication (MFA) for the user.

    This is an interactive command and will generate a QR code. You can use an authenticator app, such as Google Authenticator
    to scan the code and enter the valid six-digit verification code from the authenticator app in the interactive prompt to
    enable and activate MFA for your account.

    After three invalid attempts, you have to call this command again to generate a new QR code.
    """

    from airt.client import User

    qr = qrcode.QRCode()
    qr.add_data(User._get_mfa_provision_url(otp=otp))

    typer.echo("Please open an authenticator app and scan the QR code below:")
    #     typer.echo(qr.print_ascii(invert=True))
    typer.echo(qr.print_ascii())

    for i in range(3):

        try:
            activation_otp = typer.prompt(
                "Please enter the OTP displayed in the authenticator app"
            )
            response = User.activate_mfa(otp=activation_otp)
            typer.echo("Multi-Factor Authentication (MFA) successfully activated.")

            details = User.details()
            status = helper.get_phone_registration_status(details)
            if status is not None:
                typer.echo(status)

            break

        except ValueError as e:
            typer.echo(e)

In [ ]:
result = runner.invoke(mfa_app, ["--help"])

display(result.stdout)
assert "enable" in result.stdout

'Usage: enable [OPTIONS]\n\n  Enable Multi-Factor Authentication (MFA) for the user.\n\n  This is an interactive command and will generate a QR code. You can use an\n  authenticator app, such as Google Authenticator to scan the code and enter the\n  valid six-digit verification code from the authenticator app in the\n  interactive prompt to enable and activate MFA for your account.\n\n  After three invalid attempts, you have to call this command again to generate\n  a new QR code.\n\nOptions:\n  --otp TEXT                      Dynamically generated six-digit verification\n                                  code from the authenticator app. Please pass\n                                  this optional argument only if you have\n                                  activated the MFA for your account.\n  --install-completion [bash|zsh|fish|powershell|pwsh]\n                                  Install completion for the specified shell.\n  --show-completion [bash|zsh|fish|powershell|pwsh]\n       

In [ ]:
# | exporti


@mfa_app.command()  # type: ignore
@helper.requires_totp_or_otp(message_template_name="disable_mfa")
@helper.requires_auth_token
def disable(
    user: Optional[str] = typer.Option(
        None,
        "--user",
        "-u",
        help="Account user_uuid/username to disable MFA. If not passed, then the default value None will be used to disable MFA for the currently logged-in user.",
    ),
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app or the OTP you have received via SMS.",
    ),
) -> None:
    """Disable Multi-Factor Authentication (MFA) for the user.

    The command switches to interactive mode unless the OTP argument is passed. The interactive mode will prompt you to
    choose an OTP option you want to use. Currently, we only support disabling MFA either using a TOTP or SMS OTP.

    If you have access to the authenticator application, then you can either enter the dynamically generated six-digit
    verification code from the authenticator app (TOTP) or request an OTP via SMS to your registered phone number.

    After selecting an option, please follow the on-screen instructions to disable MFA for your account. In case,
    you don't have access to the authenticator app and your registered phone number, please contact your administrator.

    Note: Please do not pass the user argument unless you are a super user. Only
    a super user can disable MFA for other users.
    """

    from airt.client import User

    User.disable_mfa(user=user, otp=otp)

    typer.echo(
        "Multi-Factor Authentication (MFA) is successfully deactivated for the user."
    )

In [ ]:
result = runner.invoke(mfa_app, ["disable", "--help"])

display(result.stdout)
assert "disable" in result.stdout

"Usage: root disable [OPTIONS]\n\n  Disable Multi-Factor Authentication (MFA) for the user.\n\n  The command switches to interactive mode unless the OTP argument is passed.\n  The interactive mode will prompt you to  choose an OTP option you want to use.\n  Currently, we only support disabling MFA either using a TOTP or SMS OTP.\n\n  If you have access to the authenticator application, then you can either enter\n  the dynamically generated six-digit  verification code from the authenticator\n  app (TOTP) or request an OTP via SMS to your registered phone number.\n\n  After selecting an option, please follow the on-screen instructions to disable\n  MFA for your account. In case,  you don't have access to the authenticator app\n  and your registered phone number, please contact your administrator.\n\n  Note: Please do not pass the user argument unless you are a super user. Only a\n  super user can disable MFA for other users.\n\nOptions:\n  -u, --user TEXT  Account user_uuid/username to 

In [ ]:
with normal_user():
    random_otp = 123456

    # Negative Scenario. Non-MFA user passing otp
    result = runner.invoke(mfa_app, ["disable", "--otp", random_otp])

    display(result.stdout)
    assert (
        "Please pass the OTP only after activating the MFA" in result.stdout
    ), result.stdout
    assert result.exit_code == 1

'Error: MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.\n'

In [ ]:
with normal_user():
    random_otp = 123456
    result = runner.invoke(
        mfa_app, ["disable", "--user", INVALID_UUID_FOR_TESTING, "--otp", random_otp]
    )

    display(result.stdout)
    assert result.exit_code == 1
    assert (
        "Insufficient permission to access other user's data" in result.stdout
    ), result.stdout

"Error: Insufficient permission to access other user's data\n"

In [ ]:
# | exporti

# Adding mfa as a subcommand for user command
app.add_typer(mfa_app, name="mfa")

In [ ]:
result = runner.invoke(app, ["--help"])
display(result.stdout)
assert "Commands for enabling and disabling Multi-Factor" in result.stdout

result = runner.invoke(app, ["mfa", "--help"])

assert "Disable Multi-Factor Authentication (MFA) for the user" in result.stdout
display(result.stdout)

'Usage: root [OPTIONS] COMMAND [ARGS]...\n\n  A set of commands for managing users and their authentication in the server.\n\nOptions:\n  --install-completion [bash|zsh|fish|powershell|pwsh]\n                                  Install completion for the specified shell.\n  --show-completion [bash|zsh|fish|powershell|pwsh]\n                                  Show completion for the specified shell, to\n                                  copy it or customize the installation.\n  --help                          Show this message and exit.\n\nCommands:\n  create                 Create a new user in the server.\n  details                Get user details\n  disable                Disable a user in the server.\n  enable                 Enable a disabled user in the server.\n  ls                     Return the list of users available in the server.\n  mfa                    Commands for enabling and disabling Multi-Factor...\n  register-phone-number  Register and validate a phone number\n  reset-

'Usage: root mfa [OPTIONS] COMMAND [ARGS]...\n\n  Commands for enabling and disabling Multi-Factor Authentication (MFA).\n\nOptions:\n  --help  Show this message and exit.\n\nCommands:\n  disable  Disable Multi-Factor Authentication (MFA) for the user.\n  enable   Enable Multi-Factor Authentication (MFA) for the user.\n'

In [ ]:
# | exporti

sso_app = typer.Typer(help="Commands for enabling and disabling Single sign-on (SSO).")

In [ ]:
# | exporti


@sso_app.command()  # type: ignore
@helper.requires_totp()
@helper.requires_auth_token
def disable(
    sso_provider: str = typer.Argument(
        ...,
        help="Name of the Single sign-on (SSO) identity provider. At present, the API only supports Google and Github as valid SSO identity providers.",
    ),
    user: Optional[str] = typer.Option(
        None,
        "--user",
        "-u",
        help="Account user_uuid/username to disable MFA. If not passed, then the default value None will be used to disable SSO for the currently logged-in user.",
    ),
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app. Please pass this optional argument only if you have activated the MFA for your account.",
    ),
) -> None:
    """Disable Single sign-on (SSO) for the user.

    Please do not pass the user argument unless you are a super user. Only
    a super user can disable SSO for other users.
    """

    from airt.client import User

    success_msg = User.disable_sso(sso_provider=sso_provider, user=user, otp=otp)
    typer.echo(success_msg)

In [ ]:
with normal_user():
    result = runner.invoke(sso_app, ["google"])

    display(result.stdout)
    assert result.exit_code == 1

    # Negative Scenario. Non-MFA user passing otp
    random_otp = 123456
    result = runner.invoke(sso_app, ["google", "--otp", random_otp])

    display(result.stdout)
    assert (
        "Please pass the OTP only after activating the MFA for your account."
        in result.stdout
    ), result.stdout
    assert result.exit_code == 1

'Error: SSO is not enabled for the provider.\n'

'Error: MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.\n'

In [ ]:
# | exporti


@sso_app.command()  # type: ignore
@helper.requires_totp()
@helper.requires_auth_token
def enable(
    sso_provider: str = typer.Argument(
        ...,
        help="Name of the Single sign-on (SSO) identity provider. At present, the API only supports **Google** and **Github** as valid SSO identity providers.",
    ),
    sso_email: str = typer.Option(
        ...,
        "--email",
        "-e",
        help="Email id going to be used for SSO authentication.",
    ),
    otp: Optional[str] = typer.Option(
        None,
        "--otp",
        help="Dynamically generated six-digit verification code from the authenticator app. Please pass this optional argument only if you have activated the MFA for your account.",
    ),
) -> None:
    """Enable Single sign-on (SSO) for the user"""

    from airt.client import User

    success_msg = User.enable_sso(
        sso_provider=sso_provider, sso_email=sso_email, otp=otp
    )

    typer.echo(success_msg)

In [ ]:
result = runner.invoke(sso_app, ["--help"])

display(result.stdout)
assert "enable" in result.stdout

'Usage: root [OPTIONS] COMMAND [ARGS]...\n\n  Commands for enabling and disabling Single sign-on (SSO).\n\nOptions:\n  --install-completion [bash|zsh|fish|powershell|pwsh]\n                                  Install completion for the specified shell.\n  --show-completion [bash|zsh|fish|powershell|pwsh]\n                                  Show completion for the specified shell, to\n                                  copy it or customize the installation.\n  --help                          Show this message and exit.\n\nCommands:\n  disable  Disable Single sign-on (SSO) for the user.\n  enable   Enable Single sign-on (SSO) for the user\n'

In [ ]:
with normal_user():

    sso_email = "sso_email@mail.com"
    new_sso_email = "new_sso_email@mail.com"
    sso_provider = "google"

    # Positive scenario: Disabling SSO
    result = runner.invoke(sso_app, ["disable", sso_provider])

    # Positive scenario: Enabling SSO
    result = runner.invoke(sso_app, ["enable", sso_provider, "-e", new_sso_email])

    display(result.stdout)
    assert (
        f"Single sign-on (SSO) is successfully enabled for {sso_provider}"
        in result.stdout
    ), result.stdout
    assert new_sso_email in result.stdout
    assert result.exit_code == 0

    # Positive scenario: Disabling SSO
    result = runner.invoke(sso_app, ["disable", sso_provider])

    display(result.stdout)
    assert (
        f"Single sign-on (SSO) is successfully disabled for {sso_provider}"
        in result.stdout
    ), result.stdout
    assert result.exit_code == 0, result.exit_code

    # Negative Scenario. Non-MFA user passing otp
    random_otp = 123456
    result = runner.invoke(
        sso_app, ["enable", sso_provider, "-e", sso_email, "--otp", random_otp]
    )

    display(result.stdout)
    assert (
        "Please pass the OTP only after activating the MFA for your account"
        in result.stdout
    ), result.stdout
    assert result.exit_code == 1

'Single sign-on (SSO) is successfully enabled for google. Please use new_sso_email@mail.com as the email address while authenticating with google.\n'

'Single sign-on (SSO) is successfully disabled for google.\n'

'Error: MFA is not activated for the account. Please pass the OTP only after activating the MFA for your account.\n'

In [ ]:
# | exporti

# Adding sso as a subcommand for user command
app.add_typer(sso_app, name="sso")

In [ ]:
result = runner.invoke(app, ["--help"])
display(result.stdout)
assert "Commands for enabling and disabling Single sign-on" in result.stdout

result = runner.invoke(app, ["sso", "--help"])

display(result.stdout)
assert "Enable Single sign-on (SSO)" in result.stdout

'Usage: root [OPTIONS] COMMAND [ARGS]...\n\n  A set of commands for managing users and their authentication in the server.\n\nOptions:\n  --install-completion [bash|zsh|fish|powershell|pwsh]\n                                  Install completion for the specified shell.\n  --show-completion [bash|zsh|fish|powershell|pwsh]\n                                  Show completion for the specified shell, to\n                                  copy it or customize the installation.\n  --help                          Show this message and exit.\n\nCommands:\n  create                 Create a new user in the server.\n  details                Get user details\n  disable                Disable a user in the server.\n  enable                 Enable a disabled user in the server.\n  ls                     Return the list of users available in the server.\n  mfa                    Commands for enabling and disabling Multi-Factor...\n  register-phone-number  Register and validate a phone number\n  reset-

'Usage: root sso [OPTIONS] COMMAND [ARGS]...\n\n  Commands for enabling and disabling Single sign-on (SSO).\n\nOptions:\n  --help  Show this message and exit.\n\nCommands:\n  disable  Disable Single sign-on (SSO) for the user.\n  enable   Enable Single sign-on (SSO) for the user\n'